# Zillow Challenge

##  Data input

In [493]:
#### TEST MODE  #####
train = train_bak
train = train.sample(frac=1)

##  Feature creation

In [494]:
features_created = set()

In [495]:
# Types
train['propertycountylandusecode'] = pd.Categorical(train['propertycountylandusecode']).codes
train['propertyzoningdesc'] = pd.Categorical(train['propertyzoningdesc']).codes

#Binary
train['taxdelinquencyflag'] = pd.Categorical(train['taxdelinquencyflag']).codes

In [496]:
#################
#  taxrate
#################
def add_taxrate(df):
    df['taxrate'] = np.divide(df['taxamount'],df['taxvaluedollarcnt'])
    return df

features_created.add('taxrate')
train = add_taxrate(train)

In [497]:
train.columns

Index([u'logerror', u'transactiondate', u'airconditioningtypeid',
       u'architecturalstyletypeid', u'basementsqft', u'bathroomcnt',
       u'bedroomcnt', u'buildingclasstypeid', u'buildingqualitytypeid',
       u'calculatedbathnbr', u'decktypeid', u'finishedfloor1squarefeet',
       u'calculatedfinishedsquarefeet', u'finishedsquarefeet12',
       u'finishedsquarefeet13', u'finishedsquarefeet15',
       u'finishedsquarefeet50', u'finishedsquarefeet6', u'fips',
       u'fireplacecnt', u'fullbathcnt', u'garagecarcnt', u'garagetotalsqft',
       u'hashottuborspa', u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequarterbathnbr', u

Conclusions:
   * Average logerror = 0.011457219606756575
   * In general, data usually underestimates, with the etrainception of fewer high overestimates.
   * totalroomcnt:  ==0 is a mitrained bag.  != zero is overestimating

##  Feature selection

In [498]:
features_to_keep = set([
    ##########
    #  train data
    ##########
        
   #'logerror', # Predictor variable 
         
 ##########
 #  Numeric attributes
 ##########          
  #'basementsqft', #  Finished living area below or partially below ground level
  #'bathroomcnt',	#  Number of bathrooms in home including fractional bathrooms
  #'bedroomcnt',	#  Number of bedrooms in home 
  #'calculatedbathnbr',	#  Number of bathrooms in home including fractional bathroom
  #'threequarterbathnbr',	#  Number of 3/4 bathrooms in house (shower + sink + toilet)
  #'finishedfloor1squarefeet',	#  Size of the finished living area on the first (entry) floor of the home
  'calculatedfinishedsquarefeet',	#  Calculated total finished living area of the home 
  #'finishedsquarefeet6',	# Base unfinished and finished area
  #'finishedsquarefeet12',	# Finished living area
  #'finishedsquarefeet13',	# Perimeter  living area
  #'finishedsquarefeet15',	# Total area
  #'finishedsquarefeet50',	#  Size of the finished living area on the first (entry) floor of the home
 #'fireplacecnt',	#  Number of fireplaces in a home (if any)
 #'fullbathcnt',	#  Number of full bathrooms (sink, shower + bathtub, and toilet) present in home
 #'garagecarcnt',	#  Total number of garages on the lot including an attached garage
 #'garagetotalsqft',	#  Total number of square feet of all garages on lot including an attached garage
 #'lotsizesquarefeet',	#  Area of the lot in square feet
 #'numberofstories',	#  Number of stories or levels the home has
 #'poolcnt',	#  Number of pools on the lot (if any)
 #'poolsizesum',	#  Total square footage of all pools on property
 #'taxvaluedollarcnt',	# The total tax assessed value of the parcel
 #'roomcnt',	#  Total number of rooms in the principal residence
 #'unitcnt',	#  Number of units the structure is built into (i.e. 2 = dupletrain, 3 = tripletrain, etc...)
 #'yardbuildingsqft17',	# Patio in  yard
 #'yardbuildingsqft26',	# Storage shed/building in yard
 #'structuretaxvaluedollarcnt',	# The assessed value of the built structure on the parcel
 #'landtaxvaluedollarcnt',	# The assessed value of the land area of the parcel
 #'taxamount'	# The total property tatrain assessed for that assessment year
      
 ##########
 #  Time
 ##########
#####    'transactiondate',   ## from train data
 #'yearbuilt',	#  The Year the principal residence was built      
 #'taxdelinquencyyear'	# Year for which the unpaid propert tatraines were due 
 #'assessmentyear',	# The year of the property tax assessment 
     
 ##########
 #  Location
 ########## 
 #'latitude',	#  Latitude of the middle of the parcel multiplied by 10e6
 #'longitude'	#  Longitude of the middle of the parcel multiplied by 10e6        
 # 'regionidcounty',	# County in which the property is located
 #'regionidcity',	#  City in which the property is located (if any)
 #'regionidzip',	#  Zip code in which the property is located
 #'regionidneighborhood',	# Neighborhood in which the property is located
 #'rawcensustractandblock',	#  Census tract and block ID combined - also contains blockgroup assignment by etraintension
 #'censustractandblock',	#  Census tract and block ID combined - also contains blockgroup assignment by etraintension        
        
 ##########
 #  Binary
 ##########         
 #'hashottuborspa',	#  Does the home have a hot tub or spa - array([nan, True], dtype=object)
        # array([nan, True], dtype=object)
 #'pooltypeid10',	#  Spa or Hot Tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 #'pooltypeid2',	#  Pool with Spa/Hot Tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 #'pooltypeid7',	#  Pool without hot tub - [array([ nan,   1.])]
        # array([ nan,   1.])
 #'fireplaceflag',	#  Is a fireplace present in this home - array([nan, True], dtype=object)
        # array([nan, True], dtype=object)
 #'taxdelinquencyflag',	# Property tatraines for this parcel are past due as of 2015 - array([nan, 'Y'], dtype=object)
        # array([nan, 'Y'], dtype=object)
        
 ##########
 #  Types
 ##########    
 #'airconditioningtypeid', #  Type of cooling system present in the home (if any)
 #'architecturalstyletypeid', #  Architectural style of the home (i.e. ranch, colonial, split-level, etc…) 
        # array([ nan,   7.,  21.,   8.,   2.,   3.,   5.,  10.,  27.])
 #'buildingqualitytypeid',	#  Overall assessment of condition of the building from best (lowest) to worst (highest)
        # array([ nan,   7.,   4.,  10.,   1.,  12.,   8.,   3.,   6.,   9.,   5.,
        #11.,   2.])
 #'buildingclasstypeid'	# The building framing type (steel frame, wood frame, concrete/brick) 
        # array([ nan,   3.,   4.,   5.,   2.,   1.])
 #'decktypeid',	# Type of deck (if any) present on parcel
        # array([ nan,  66.])
 #'heatingorsystemtypeid',	#  Type of home heating system
        # array([ nan,   2.,   7.,  20.,   6.,  13.,  18.,  24.,  12.,  10.,   1.,
        # 14.,  21.,  11.,  19.])
 'propertycountylandusecode',	#  County land use code i.e. it's zoning at the county level
        #array([nan,'010D', '0109', '1200', '1210', '010V', '300V', '0100', '0200',
        #'010C', '0300', '1204', '100V', '01HC', '120C', '040V', '1214', ... 
        # ], dtype=object)
 #'propertylandusetypeid',	#  Type of land use the property is zoned for
        # array([ 269.,  261.,   47.,   31.,  260.,  266.,  246.,  247.,  248.,
        # 263.,  265.,  275.,  267.,  264.,  270.,   nan])
 'propertyzoningdesc',	#  Description of the allowed land uses (zoning) for that property   
        # array([nan, 'LCA11*', 'LAC2', ..., 'WCR1400000', 'EMPYYY', 'RMM2*'], dtype=object)
 #'typeconstructiontypeid',	#  What type of construction material was used to construct the home
        # array([ nan,   6.,   4.,  10.,  13.,  11.])
 #'storytypeid',	#  Type of floors in a multi-story house (i.e. basement and main level, split-level, attic, etc.).  See tab for details.
        # array([ nan,   7.])

 ##########
 #  Misc
 ########## 
 #'fips',	#  Federal Information Processing Standard code -  see https://en.wikipedia.org/wiki/FIPS_county_code for more details
        # array([ 6037.,  6059.,  6111.,    nan])
 #'parcelid',	#  Unique identifier for parcels (lots)   
])

features_to_keep = features_to_keep.union(features_created)

In [499]:
features_to_keep

{'calculatedfinishedsquarefeet',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'taxrate'}

In [500]:
features_complete = train.columns[list(~train.isnull().any().values)]
features_complete

Index([u'logerror', u'transactiondate', u'bathroomcnt', u'bedroomcnt', u'fips',
       u'latitude', u'longitude', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcounty', u'roomcnt',
       u'assessmentyear', u'taxdelinquencyflag'],
      dtype='object')

In [501]:
train=train.fillna(-1)

In [502]:
Y = train[['logerror']]
X = train
for column in X.columns:
    if column not in features_to_keep:
        X = X.drop(column, axis=1)
        
#X = pd.DataFrame(scale(train, axis=0, with_mean=True, with_std=True, copy=True), columns = X.columns.values)

In [503]:
# TODO: qualifiers
#X = X.drop('propertylandusetypeid', axis =1)
#X = X.drop('rawcensustractandblock', axis =1)
#X = X.drop('regionidcounty', axis =1)

In [504]:
X.describe()

,calculatedfinishedsquarefeet,propertycountylandusecode,propertyzoningdesc,taxrate
count,"90,275.00","90,275.00","90,275.00","90,275.00"
mean,"1,760.20",24.32,570.64,0.02
std,937.05,25.74,574.12,0.44
min,-1.00,-1.00,-1.00,-1.00
25%,"1,176.00",2.00,-1.00,0.01
50%,"1,535.00",9.00,609.00,0.01
75%,"2,089.00",55.00,835.00,0.01
max,"22,741.00",76.00,"1,995.00",109.54


In [505]:
Y.describe()

,logerror
count,"90,275.00"
mean,0.01
std,0.16
min,-4.61
25%,-0.03
50%,0.01
75%,0.04
max,4.74


## Modeling

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.9)

In [ ]:
# Fit regression model
#max_depth = None
#min_samples_split = int(round(len(X_train)/50, 0))
#min_samples_leaf = int(round(len(X_train)/10, 0))
max_depth = 100
min_samples_split = 1000
min_samples_leaf = 100

print("min_samples_split =", min_samples_split)
print("min_samples_split =", min_samples_leaf)

model = DecisionTreeRegressor(max_depth=max_depth, 
                              min_samples_split = min_samples_split,
                              min_samples_leaf = min_samples_leaf,
                              criterion = "mae")
model.fit(X_train, Y_train)

('min_samples_split =', 1000)
('min_samples_split =', 100)


In [ ]:
Y_predict=model.predict(X_test)

In [ ]:
# (<= value) left = false, right = true
zip(X.columns[model.tree_.feature], model.tree_.threshold, model.tree_.children_left, model.tree_.children_right)

In [ ]:
#Decision_Tree_Image(model, X_train.columns)

## Model evaluation

In [ ]:
# Zillow MAE (test)
print("MAE: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

In [ ]:
print ("Features: ", X.columns)
print ("Tree params: ")
print ("   max_depth ", max_depth)
print ("   min_samples_split ", min_samples_split)
print ("   min_samples_leaf ", min_samples_leaf)
print ("Result: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))

Results:

Features: calculatedfinishedsquarefeet
Tree params:
    max_depth: 200
    min_samples_split: 500
    min_samples_leaf: 5
Result: ('MAE: ', 0.067769827204253438)

('Features: ', Index([u'bathroomcnt', u'bedroomcnt', u'calculatedfinishedsquarefeet',
       u'roomcnt', u'taxrate'],
      dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.068984714222419155)


('Features: ', Index([u'calculatedfinishedsquarefeet', u'taxrate'], dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.06646845923792645)


('Features: ', Index([u'calculatedfinishedsquarefeet', u'taxrate', u'roomcnt_offby'], dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.068475326761187422)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'roomcnt', u'taxrate',
       u'roomcnt_offby'],
      dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.06741981058927779)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'roomcnt', u'taxrate'], dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.067915003322995146)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'roomcnt', u'taxrate',
       u'logerroravg - zip'],
      dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.069011060035445271)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'roomcnt', u'taxrate',
       u'logerroravg - neighborhood'],
      dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 500)
('   max_samples_leaf ', 5)
('Result: ', 0.068019217988480277)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'taxrate'], dtype='object'))
Tree params: 
('   max_depth ', 200)
('   min_samples_split ', 1000)
('   max_samples_leaf ', 100)
('Result: ', 0.067869511519716444)


('Features: ', Index([u'calculatedfinishedsquarefeet', u'propertycountylandusecode',
       u'taxrate'],
      dtype='object'))
Tree params: 
('   max_depth ', 100)
('   min_samples_split ', 1000)
('   max_samples_leaf ', 100)
('Result: ', 0.068272928666371302)

('Features: ', Index([u'calculatedfinishedsquarefeet', u'propertycountylandusecode',
       u'taxdelinquencyflag', u'taxrate'],
      dtype='object'))
Tree params: 
('   max_depth ', 100)
('   min_samples_split ', 1000)
('   max_samples_leaf ', 100)
('Result: ', 0.070624357554275596)

## Model Execution

In [469]:
sample_submission_f = '/Users/Jose/Desktop/Zillow/sample_submission.csv'
submission = pd.read_csv(sample_submission_f, index_col='ParcelId')

In [470]:
#  ATTEMPT 2 - 
logerroravg = train['logerror'].mean()
print("Using logerroravg = ", logerroravg)

('Using logerroravg = ', 0.011457219606756525)


In [546]:
X_all = properties

In [ ]:
# Prediction transaction date
def add_transactiondate(df):
    df['transactiondate'] = pd.datetime(2016,10,1)
    df['transactiondate'] = df['transactiondate'].astype(int)

add_transactiondate(X_all)

In [47]:
# Add features


In [ ]:
for column in X_all.columns:
    if column not in features_to_keep:
        X_all = X_all.drop(column, axis=1)

In [547]:
#  Check for missing data
print("Missing: ",X_test.columns.difference( X_all.columns))

('Missing: ', Index([u'distanceavg', u'finishedsquarefeet12_t', u'finishedsquarefeet13_t',
       u'finishedsquarefeet15_t', u'finishedsquarefeet50_t',
       u'finishedsquarefeet6_t', u'regionidzip_avg', u'transactiondate'],
      dtype='object'))


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  from ipykernel import kernelapp as app


In [534]:
Y_all = model.predict(X_all)

In [ ]:
results = pd.DataFrame(index=X_all.index)
results.index.names = ['ParcelId']
results['201610'] = Y_all
results['201611'] = Y_all
results['201612'] = Y_all
results['201710'] = Y_all
results['201711'] = Y_all
results['201712'] = Y_all

In [ ]:
submission = submission.drop(submission.columns[0:], axis=1)
submission = submission.join(results)

# Use average for properties with missing data
submission.fillna(logerroravg, inplace=True)       
submission.columns

In [ ]:
# Round as per rules
submission = submission.round(4)

## Sanity Checks

In [ ]:
# Average log error
submission.describe().round(4)

In [ ]:
# Check for NaN
submission.isnull().sum()

In [ ]:
# Check if any duplicates
submission[submission.index.duplicated(keep=False)]

In [ ]:
# Check additional values in submission file
submission[~submission.index.isin(properties.index)]

In [ ]:
# Check additional values in properties file
properties[~properties.index.isin(submission.index)]

## Dump File

In [ ]:
# Write file
submission_f = '/Users/Jose/Desktop/Zillow/submission.csv'
submission_fh = open(submission_f, 'wb')
submission.to_csv(submission_fh, sep=',', header='true')
submission_fh.close()

## Historical Records

## Resources

In [ ]:
# bedroomcnt
#plt.trainlabel('bedroomcnt')
#plt.ylabel('logerror')
#plt.scatter(train['bathroomcnt'], train['logerror'], 
#           alpha=1, s=2, color='r')
#plt.atrainis([0, 10,-5, 5])
#plt.show()

#date
#plt.trainlabel('transactiondate')
#plt.ylabel('logerror')
#plt.scatter(train['bedroomcnt'], train['logerror'], alpha=1, s=2, color='r')
#plt.atrainis(['2016-01-01', '2017-01-01',-5, 5])
#plt.gcf().autofmt_traindate()
#plt.show()

#import matplotlib as mpl
#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train['longitude'], train['latitude'], c=train['logerror'], s=20, lw=0, cmap='seismic')
#plt.atrainis([min(train['longitude']), matrain(train['longitude']),min(train['latitude']), matrain(train['latitude'])])
#plt.colorbar()
#plt.show()

#train2 = train.query('logerror < -2.')

#train2 = train.query('roomcnt == 0 ')

#train2 = train.query('bedroomcnt > 2')

#train2 = train.sample(frac=.10)

#plt.trainlabel('longitude')
#plt.ylabel('latitude')
#plt.scatter(train2['longitude'], train2['latitude'], c=train2['logerror'], s=20, lw=0, cmap='seismic')
#plt.colorbar()
#plt.show()